In [1]:
%load_ext autoreload
%autoreload 2 
# !apt-get install -y xvfb
import time
import torch
import scipy
import scipy.sparse
from collections import Counter
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.data import Dataset
# https://www.youtube.com/watch?v=QLIkOtKS4os --> creating custom dataset in pytorch geometric
from torch.utils.data import Dataset, random_split
import torch_geometric
from torch_geometric.data import Data, InMemoryDataset
import torch_geometric
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GlobalAttention, SAGEConv

import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.utils import to_networkx, from_networkx
from sklearn.model_selection import StratifiedKFold
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import pickle
import seaborn as sn
import random
import os
from typing import Optional

from torch_scatter import scatter_add

from torch_geometric.utils import softmax

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

from graph_utils import set_device_and_seed, set_seed, show, visualize_graph, visualize_embedding, _count_parameters, visualise_airway_tree_matplotlib
from graph_datasets import CustomDataset
from graph_models import CustomGlobalAttention, GAT
from graph_training import train_model, test_model, _vis_graph_example, train_test_split

In [3]:
def dice_loss(input, target):
    smooth = 1.

    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return 1 - ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))



In [25]:


class CustomDatasetNode(torch_geometric.data.Dataset):
    def __init__(self,
                 root,
                 filename_data,
                 filename_labels,
                 test=False,
                 transform=None,
                 pre_transform=None,
                 label_col_name = 'binaryLL_1',
                 node_level = True,
                args = {'node_feature_names': [], 'edge_feature_names': []}):
        '''
        For NODE LEVEL CLASSIFICATION
        root = where dataset should be stored, folder is split into raw_dir and processed_dir
        filename_data = contains X features for nodes + edges (csv)
        filename_labels= contains Y labels for graphs (csv)
        
        
        '''

            
            
        self.test = False
        self.filename_data = os.path.abspath(filename_data)
        self.filename_labels = os.path.abspath(filename_labels)
        self.node_map = {}
        self.node_level = node_level
        self.y = None
        #         super(CustomDataset, self).__init__(root, transform, pre_transform)
        if len(args['node_feature_names']) > 0:
            self.node_feature_names = args['node_feature_names']
        else:
            self.node_feature_names = None
        
        if len(args['edge_feature_names']) > 0:
            self.edge_feature_names = args['edge_feature_names']
        else:
            self.edge_feature_names = None
        print(f"Using Node features: {self.node_feature_names}, Edge features: {self.edge_feature_names}")
        self.label_col = label_col_name
        print(f"Getting labels from: {self.label_col}")
        super(CustomDatasetNode, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        return self.filename_data
    
    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped: NOTE NOT SURE WHAT THIS IS"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
    
    def _download(self):
        pass
    
    def process(self):
        self.data = pd.read_csv(os.path.abspath(self.raw_paths[0]))
        label_df = self._process_labels()
        graph_ids = self.data.idno.unique()
        
        for i, idno in tqdm(list(enumerate(graph_ids))):
            # iterate through each patricipant (resetindex is important for node relabelling)
            df = self.data.loc[self.data.idno == idno].copy().reset_index()
            # build a graph out of the df with node features, edge_features and edge_adjacency
            x = self._get_node_features(df)
            edge_adjacency = self._get_edge_adjacency(df,index=i)
            edge_features = self._get_edge_features(df)
            y = self._get_label(idno, label_df, x)
            data = Data(x=x,
                        edge_index=edge_adjacency,
                        edge_attr = edge_features,
                        y = y
                       )
            if self.test:
                torch.save(data, 
                            os.path.join(self.processed_dir, 
                                         f'data_test_{i}.pt'))
            else:
                 torch.save(data, 
                            os.path.join(self.processed_dir, 
                                         f'data_{i}.pt'))
    
    
    def _process_labels(self):
        '''
        Reads label df - checks its binary 0,1 labels (REQUIRES COLUMN self.label_col) if not returns an error 
        Keeps only rows that match idnos in self.data_df which is processed first
        measures class proportions and saves to self.class_proportions
        returns a df which has a label per idno with label in col called 'y'
        '''
        label_df = pd.read_csv(os.path.abspath(self.filename_labels))
        # binarise 
        assert self.label_col in label_df.columns, f"The column {self.label_col} cannot be found"
        # drop unnecessary cols
        
        # drop rows not matching to data ids
        data_df =  pd.read_csv(os.path.abspath(self.filename_data))
        label_df_small = label_df.loc[label_df.idno.isin(data_df.idno.unique())]
        self.y = label_df_small
        print("# Graphs", len(label_df_small), "Label Frequency", Counter(label_df_small[self.label_col].to_list()))
        self.class_proportions = {k:v/len(label_df_small) for k,v in Counter(label_df_small[self.label_col].to_list()).items()}
        print(f"Class proportions: {self.class_proportions}")
        return label_df_small
    
    def _get_edge_adjacency(self,df, index):
        '''
        NOTE pytorch requires nodes to start from 0 and go up in integers so need to remap start and end bpids from df
        Turns endbpid 1 > 1, if the next one in the df is endbpid=6 it becomes 2 etc so that the endbpids are in order from 1 to max (index+1)
        Adds trachea (node 0)
        applyys relabelling to start and endbpid based on the dict 
        returns torch tensor in COO format which is a parallel list [[source_node_list], [corresponding_end_node_list]]
        in this format, at list[0][2] and list[1][2] will be source-end node of the second edge in the list
        '''
        # reindex start / endbpids
        
        relabel_map = {v:k+1 for k,v in df.endbpid.to_dict().items()}
        # add trachea map (node 0)
        relabel_map[-1] = 0
        # save mapping
        self.node_map[index] = (df.idno.unique().item(), relabel_map)
        # apply relabelling to source and end nodes
        source_nodes = df.startbpid.apply(lambda x: relabel_map[x]).to_list()
        end_nodes = df.endbpid.apply(lambda x: relabel_map[x]).to_list()
        # return in COO format
        return torch.tensor([source_nodes, end_nodes], dtype=torch.long)

    def _get_node_features(self,df):
        '''
        DF already contains normalised features
        Feature names in node_features
        Format to return is a tensor of shape num_nodes x node_feature_dim with dtype float 
        assumes ordered in ascending order with nodes 0 and up in integers (so node = index / row num)
            node_features = ['x_norm', 'y_norm', 'z_norm', 'dircosx_norm',
       'dircosy_norm', 'dircosz_norm','lobe_norm',
       'sublobe_norm','angle_norm', 'weibel_generation_norm','dist_nn_in_lobe_norm', 'num_desc_norm','max_path_length_norm']

        '''
        # nodes and features (pos, direction,lobe + sublobe categorical, angle to parent, weibel gen, dist to nearest neighbor in lobe, max path length to leaf, num descendents of node)
        node_features = self.node_feature_names

        # adding trachea info to top of list
        trachea_dict = dict.fromkeys(node_features, 0)
        for i in ['nx', 'ny', 'nz']:
            trachea_dict[i] = df.loc[df.endbpid==1][str('parent_loc_'+i)].item()
        
        # currently usnig as features 
        list_of_nodes = df[node_features].to_dict(orient='records')
        # add trachea to nodes
        list_of_lists_nodes =[list(trachea_dict.values())]+ [list(node_feature.values()) for node_feature in list_of_nodes]
        x = torch.tensor(list_of_lists_nodes, dtype=torch.float)
#         print("Shape node features", x.shape)
        return x

    def _get_edge_features(self,df):
        '''
        get matrix of shape [# edges, edge feature size] with type float
        '''
        if self.edge_feature_names is not None:
            edge_feature_names = self.edge_feature_names
        else:
            edge_feature_names = ['centerlinelength_norm','avginnerarea_norm']
            
        edge_norm = df[edge_feature_names].values
        return torch.tensor(edge_norm, dtype=torch.float)

    def _get_label(self, idno, label_df,node_features):
        '''
        for the selected idno, returns value in self.label_col as an integer
        '''
        # Assume trachea node is always non anomalous (0)
        if self.node_level:
            labels = label_df.loc[label_df.idno==idno,
                 self.label_col].values
            
            num_nodes = node_features.shape[0]
            if num_nodes -1 == labels.shape[0]:
#                 print('adding in label for node -1 = 0 in pytorch method')
                labels = [0] + list(labels)
            
#                 print('Num nodes, num labels', num_nodes, len(labels))
            
            return torch.tensor(labels, dtype=torch.int64)
        else:
            # return single graph label
            labels = label_df.loc[label_df.idno==idno,
                 self.label_col].unique()
#             print('num labels per graph', labels.shape[0])
        
            return torch.tensor(labels, dtype=torch.int64)
    
    def len(self):
        return int(self.data.idno.nunique())
    
    def get(self, idx):
        '''
        Equivalent to __getitem__ in pytorch
        '''
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))   
        return data
            

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)

In [24]:
label_col_name='node_label'

#  removed from features for toy example, added edge features to node ones too just in case
node_features = ['nx', 'ny', 'nz', 'dircosx_norm',
               'dircosy_norm', 'dircosz_norm','angle_norm', 'weibel_generation_norm','dist_nn_in_lobe_norm',
                 'num_desc_norm','max_path_length_norm','centerlinelength_norm','avginnerarea_norm',
                'lobe_norm', 'num_children']
edge_feature_names = ['centerlinelength_norm','avginnerarea_norm']

args = {'node_feature_names': node_features, 'edge_feature_names':edge_feature_names}
# DATASETS


my_data_train  = CustomDatasetNode('data_train_toy/',
                               '/home/sneha/toy_normalised_1407.csv',
                               "/home/sneha/toy_labels_1407.csv",
                               args = args,
                               label_col_name=label_col_name,
                                node_level=True
                              )

Using Node features: ['nx', 'ny', 'nz', 'dircosx_norm', 'dircosy_norm', 'dircosz_norm', 'angle_norm', 'weibel_generation_norm', 'dist_nn_in_lobe_norm', 'num_desc_norm', 'max_path_length_norm', 'centerlinelength_norm', 'avginnerarea_norm', 'lobe_norm', 'num_children'], Edge features: ['centerlinelength_norm', 'avginnerarea_norm']
Getting labels from: node_label


Processing...


# Graphs 120976 Label Frequency Counter({0: 103792, 1: 17184})
Class proportions: {0: 0.857955296918397, 1: 0.14204470308160297}


  0%|▎                                                                                 | 9/2093 [00:00<00:24, 85.72it/s]

adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 49 49
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 73 73
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 50 50
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 64 64
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 71 71
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 51 51
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 52 52
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 50 50
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 68 68
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 49 49
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 64 64
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 67 67
adding in label for node -1 = 0 in pytorch method
Num nodes, num

...

100%|██████████████████████████████████████████████████████████████████████████████▋| 2085/2093 [00:21<00:00, 94.64it/s]

adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 50 50
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 52 52
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 51 51
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 52 52
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 65 65
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 50 50
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 51 51
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 52 52
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 67 67
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 69 69
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 69 69
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 70 70
adding in label for node -1 = 0 in pytorch method
Num nodes, num

100%|███████████████████████████████████████████████████████████████████████████████| 2093/2093 [00:21<00:00, 96.39it/s]

adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 45 45
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 64 64
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 67 67
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 50 50
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 67 67
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 47 47
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 51 51
adding in label for node -1 = 0 in pytorch method
Num nodes, num labels 52 52



Done!


In [32]:
x = my_data_train[0].x

x.min(), x.max(), x.shape

(tensor(-4.1278), tensor(9.4538), torch.Size([49, 15]))